In [1]:
#| include: false

import os
import pandas as pd
import numpy as np
# import plotly.express as px
import toml
# import psrc_theme
from pathlib import Path
from scipy.spatial import cKDTree
import summary_data
import sqlite3

# to show plotly figures in quarto HTML file
# import plotly.io as pio
# pio.renderers.default = "plotly_mimetype+notebook_connected"
# pio.templates.default = "simple_white+psrc_color" # set plotly template

config = summary_data.CONFIG
valid_config = toml.load(Path(Path.cwd(), '..\\..\\..\\..\\configuration', 'validation_configuration.toml'))
all_runs = summary_data.ALL_RUNS

## transit boardings

### boardings by mode

In [2]:
# all_runs['current run']

In [3]:
# Load model results and calculate modeled daily boarding by line
df = summary_data.load_agg_data('transit/transit_line_results.csv')
df_transit_line = (
    df.groupby(['source', 'route_code', 'mode', 'agency_code'])
    .agg({"description": "first", "boardings": "sum"})
    .reset_index()
)
df_transit_line['agency'] = df_transit_line['agency_code'].astype('int').astype('str').map(config['agency_lookup'])

In [4]:
tab = df_transit_line.groupby(['source','mode'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='mode')['boardings'].\
        rename_axis(columns={'mode': 'Transit boardings by mode'})
tab['Total'] = tab.sum(axis=1)

tab.rename(columns={'b': 'Bus', 'c': 'Commuter Rail', 'f': 'Ferry (f)', 'r': 'Light Rail', 'p': 'Ferry (p)'}, inplace=True)

display(tab.style.format('{:,.0f}'))


Transit boardings by mode,Bus,Commuter Rail,Ferry (f),Ferry (p),Light Rail,Total
source,,,,,,
2050 RTP,"1,454,485","31,477","51,462","12,512","783,454","2,333,391"
2050 new LU,"997,148","9,358","27,887","8,230","426,616","1,469,239"
"2050 new LU, less WFH, no telecommute","943,251","9,396","29,321","9,122","421,104","1,412,193"
2050 old LU,"960,127","10,135","31,314","9,587","436,816","1,447,979"
current run,"374,074","6,297","8,363","3,855","92,876","485,465"


### boardings by agency

In [5]:

tab = df_transit_line.groupby(['source','agency'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='agency')['boardings'].\
        rename_axis(columns={'agency': 'Transit boardings by agency'})
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))

Transit boardings by agency,Community Transit,Everett Transit,King County Metro,Kitsap Transit,Pierce Transit,Sound Transit,Washington Ferries,Total
source,,,,,,,,
2050 RTP,"153,349","24,151","1,125,607","49,424","90,672","870,358","19,830","2,333,391"
2050 new LU,"134,200","26,266","743,190","32,239","53,695","465,729","13,919","1,469,239"
"2050 new LU, less WFH, no telecommute","123,936","24,076","703,533","33,192","50,612","461,202","15,642","1,412,193"
2050 old LU,"125,257","24,713","715,945","34,487","52,802","478,346","16,430","1,447,979"
current run,"26,741","4,476","276,394","10,309","21,720","140,081","5,744","485,465"


In [6]:
df = df_transit_line[df_transit_line['mode'].isin(['b'])].copy()
tab = df.groupby(['source','agency'])[['boardings']].sum().\
        reset_index().\
        pivot(index='source', columns='agency')['boardings'].\
        rename_axis(columns={'agency': 'Transit boardings by agency'})
tab['Total'] = tab.sum(axis=1)

display(tab.style.format('{:,.0f}'))

Transit boardings by agency,Community Transit,Everett Transit,King County Metro,Kitsap Transit,Pierce Transit,Sound Transit,Total
source,,,,,,,
2050 RTP,"153,349","24,151","1,098,651","32,235","90,672","55,427","1,454,485"
2050 new LU,"134,200","26,266","732,825","20,407","53,695","29,756","997,148"
"2050 new LU, less WFH, no telecommute","123,936","24,076","693,134","20,790","50,612","30,703","943,251"
2050 old LU,"125,257","24,713","704,473","21,488","52,802","31,394","960,127"
current run,"26,741","4,476","273,489","4,909","21,720","42,739","374,074"


In [7]:
### boardings by key routes

# def show_key_route_boardings(route_list):
#         df = df_transit_line[df_transit_line['route_code'].isin(route_list)].copy()
#         tab = df.groupby(['source','description'])[['boardings']].sum().\
#                 reset_index().\
#                 pivot(index='source', columns='description')['boardings'].\
#                 rename_axis(columns={'description': 'Transit boardings by key routes'})
#         tab['Total'] = tab.sum(axis=1)

#         display(tab.style.format('{:,.0f}', na_rep="-"))

In [8]:
# # Ferry
# show_key_route_boardings([4200,4201,4203,4204,5001,5002,5003,5004,5005,5006,5007,1973,1975])
# # Rail
# show_key_route_boardings([6996,6999,6998])
# # RapidRide, busiest buses, streetcar
# show_key_route_boardings([1671,1672,1673,1674,1675,1676,6550,1007,1040,1997,1998])

In [9]:
# Light rail boardings by station
df_boardings = summary_data.load_agg_data('transit/boardings_by_stop.csv')

df_transit_stops = summary_data.load_agg_data('../inputs/scenario/networks/transit_stops.csv')
light_rail_df = df_transit_stops[df_transit_stops['light_rail']==1]
light_rail_df = light_rail_df[['PSRCJunctID', 'x', 'y','source']]

df = pd.merge(df_boardings, light_rail_df, left_on=['i_node','source'], right_on=['PSRCJunctID','source'])

# # # Load station names
df_sql = summary_data.load_sqlite("SELECT station_name, emme_node FROM light_rail_station_boardings")
# df_sql
# # # Merge station names to df 
df = df.merge(df_sql, left_on=['PSRCJunctID','source'], right_on=['emme_node','source'])
# # df.to_clipboard()
# # df_sql
df = df[['total_boardings','station_name','PSRCJunctID','source']].drop_duplicates()

# pivot table of boardings by station and source
df = df.pivot_table(index='station_name', columns='source', values='total_boardings', aggfunc='sum')
df.style.format('{:,.0f}', na_rep='-')

source,2050 RTP,2050 new LU,"2050 new LU, less WFH, no telecommute",2050 old LU,current run
station_name,,,,,
130th St Station,"5,502","3,687","3,491","3,633",-
145th St Station,"7,617","3,814","5,774","5,828",-
185th St Station,"9,192","5,675","7,764","7,043",-
6th Ave,"1,662","1,111","1,031","1,136",-
Alderwood Station,"7,079","4,811","4,460","4,334",-
Angle Lake,"9,474","4,623","3,666","3,927","2,962"
Ash Way Station,"9,902","5,789","6,113","6,473",-
Avalon Station,"3,455","2,418","2,300","2,219",-
Ballard Station,"9,748","5,222","5,121","5,428",-


## Employment within 1 mile of Light Rail Station

In [10]:
df_transit_stops = summary_data.load_agg_data('../inputs/scenario/networks/transit_stops.csv')
light_rail_df = df_transit_stops[df_transit_stops['light_rail']==1]
light_rail_df = light_rail_df[['PSRCJunctID', 'x', 'y','source']]

# light_rail_df

output_df = pd.DataFrame()

df_parcels = summary_data.load_landuse('landuse/buffered_parcels.txt')

# Create a KDTree for fast nearest-neighbor lookup
for source in df_parcels.source.unique():
    
    station_df = light_rail_df[light_rail_df['source']==source].copy()
    parcel_df = df_parcels[df_parcels['source']==source].copy()

    parcel_tree = cKDTree(parcel_df[['xcoord_p', 'ycoord_p']])

    # Query the nearest parcel for each light rail station
    distances, indices = parcel_tree.query(station_df[['x', 'y']])

    # Add the nearest parcel ID to the light rail dataframe
    station_df['nearest_parcel_id'] = parcel_df.iloc[indices]['parcelid'].values

    # Merge light_rail_df with df_parcel to get the emptot_2 field
    result_df = station_df.merge(parcel_df[['parcelid', 'emptot_2']], 
                                                 left_on='nearest_parcel_id', right_on='parcelid', how='left')


    # # Get light rail station names
    # # load from sqlite db

    con = sqlite3.connect(os.path.join(all_runs[source], 'inputs', 'db', 'soundcast_inputs_2023.db'))
    df_station_names = pd.read_sql_query("SELECT * FROM light_rail_station_boardings", con)
    df_station_names = df_station_names[['emme_node','station_name']].drop_duplicates()

    result_df = result_df.merge(df_station_names, left_on='PSRCJunctID', right_on='emme_node', how='left')

    # Select relevant columns to display
    result_df = result_df[['station_name', 'nearest_parcel_id', 'emptot_2']]

    result_df['source'] = source
    output_df = pd.concat([output_df, result_df])


In [11]:
# pivot table of emploment station name
df = output_df.groupby(['source','station_name'])[['emptot_2']].sum().\
        reset_index().\
        pivot(index='source', columns='station_name')['emptot_2']
df.T.style.format('{:,.0f}', na_rep='-')

source,2050 RTP,2050 new LU,"2050 new LU, less WFH, no telecommute",2050 old LU,current run
station_name,,,,,
130th St Station,"1,620","1,737","2,138","1,950",-
145th St Station,658,"1,232","2,406","2,002",-
185th St Station,"1,615","1,673","3,231","2,989",-
6th Ave,"38,581","38,444","39,867","39,361",-
Alderwood Station,"13,924","13,622","12,008","10,822",-
Angle Lake,"19,109","17,402","16,561","19,162","10,763"
Ash Way Station,"2,871","3,611","3,023","2,931",-
Avalon Station,"5,476","5,775","6,126","5,805",-
Ballard Station,"13,211","12,671","11,370","12,014",-


In [12]:
# # Check buffered parcels distance to light rail
# df_buffered = summary_data.load_landuse('landuse/buffered_parcels.txt')

# # Merge geography to file
# df_buffered = df_buffered.merge(df_geog_lookup, left_on='parcelid', right_on='ParcelID', how='left')


# df_buffered.groupby(['source','district_name'])['dist_lbus'].describe()
# # df_buffered.columns